Notebook to compute the nn distance between latent spaces.

In [1]:
import os
import torch
import json
import numpy as np
import numpy.random as rd
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')

from pandas.plotting import scatter_matrix
from scipy.spatial import distance
from scipy.stats import percentileofscore

from contrastive.utils.nn_distance import *

## Load the embeddings

In [2]:
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)

print(embeddings1.head())

embeddings2 = -embeddings1
embeddings2.head()

                    dim1        dim2        dim3        dim4        dim5  \
Subject                                                                    
1-03HUBJO_t0  -53.579821  283.674235 -289.220584  -39.377640   31.373385   
1-08ANDTI_t0  304.421732  221.746751  212.028836   33.319015 -129.834469   
1-11LEBJO_t0  229.580833  287.437274   26.512003 -120.822792   20.726943   
1-15LEHMI_t0  111.156533    3.509716  -62.315849  -17.678537 -288.433520   
1-17COLMA_t0 -155.806756 -150.651878 -204.962518 -183.430474 -253.461319   

                    dim6        dim7        dim8        dim9       dim10  ...  \
Subject                                                                   ...   
1-03HUBJO_t0 -158.339627    9.670317  152.284108  -65.771396 -149.046171  ...   
1-08ANDTI_t0   21.031527  119.441280  -10.692840   71.951305  124.713433  ...   
1-11LEBJO_t0    5.467387  187.373954  122.830636  -23.879877  -63.080933  ...   
1-15LEHMI_t0    9.222002  -45.920334  106.299119  199.909948  

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim21,dim22,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30
Subject,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,53.579821,-283.674235,289.220584,39.377640,-31.373385,158.339627,-9.670317,-152.284108,65.771396,149.046171,...,-87.509182,-53.117284,24.258174,48.823370,-3.300542,-7.509744,63.925171,72.504643,-102.564855,89.569466
1-08ANDTI_t0,-304.421732,-221.746751,-212.028836,-33.319015,129.834469,-21.031527,-119.441280,10.692840,-71.951305,-124.713433,...,22.659114,-77.277498,-63.190227,85.281956,116.815156,28.785544,19.209134,-141.514044,138.735023,57.747222
1-11LEBJO_t0,-229.580833,-287.437274,-26.512003,120.822792,-20.726943,-5.467387,-187.373954,-122.830636,23.879877,63.080933,...,50.180483,-72.835848,-171.984233,119.051368,50.734883,-130.860103,-42.828971,141.733466,123.745309,17.192685
1-15LEHMI_t0,-111.156533,-3.509716,62.315849,17.678537,288.433520,-9.222002,45.920334,-106.299119,-199.909948,-192.257399,...,37.666366,51.478220,109.851240,106.756577,60.081274,-94.729260,40.706943,17.842731,84.573856,24.888063
1-17COLMA_t0,155.806756,150.651878,204.962518,183.430474,253.461319,89.578872,43.501400,11.313596,-115.759039,-53.291902,...,63.945028,59.527924,44.080342,7.834308,-55.770208,86.571211,54.449158,69.077240,24.373003,-94.710483


In [3]:
embeddings1.loc['1-17COLMA_t0', :]

dim1    -155.806756
dim2    -150.651878
dim3    -204.962518
dim4    -183.430474
dim5    -253.461319
dim6     -89.578872
dim7     -43.501400
dim8     -11.313596
dim9     115.759039
dim10     53.291902
dim11   -199.851824
dim12    -38.700631
dim13     67.739853
dim14    -59.144676
dim15    179.745536
dim16     37.512961
dim17    -53.069641
dim18    110.252272
dim19    129.498434
dim20     66.243626
dim21    -63.945028
dim22    -59.527924
dim23    -44.080342
dim24     -7.834308
dim25     55.770208
dim26    -86.571211
dim27    -54.449158
dim28    -69.077240
dim29    -24.373003
dim30     94.710483
Name: 1-17COLMA_t0, dtype: float64

In [4]:
# get the subject list

# small check before
if embeddings1.index.all() != embeddings2.index.all():
    raise ValueError("The two spaces don't contain the same subjects")
else:
    list_subjects = embeddings1.index.astype('str')

list_subjects

Index(['1-03HUBJO_t0', '1-08ANDTI_t0', '1-11LEBJO_t0', '1-15LEHMI_t0',
       '1-17COLMA_t0', '1-18ROUCO_t0', '1-20FORHU_t0', '1-23CREES_t0',
       '1-40AYMJU_t0', '2-02FERMA_t0',
       ...
       'sub-inhibition20_ses-pretraining_T1w',
       'sub-inhibition21_ses-pretraining_T1w',
       'sub-inhibition22_ses-pretraining_T1w',
       'sub-inhibition23_ses-pretraining_T1w',
       'sub-inhibition24_ses-pretraining_T1w',
       'sub-inhibition25_ses-pretraining_T1w',
       'sub-inhibition26_ses-pretraining_T1w',
       'sub-inhibition28_ses-pretraining_T1w',
       'sub-inhibition29_ses-pretraining_T1w',
       'sub-inhibition31_ses-pretraining_T1w'],
      dtype='object', name='Subject', length=341)

## Compute the space distances

In [6]:
dist_mat_1 = get_distance_matrix(embeddings1)
dist_mat_2 = get_distance_matrix(embeddings2)

In [7]:
dist_mat_1['1-03HUBJO_t0'].sort_values().index

Index(['1-03HUBJO_t0', 'e0071_t1_s03', 'nih_chp_01746_t1', 'nih_chp_05041_t1',
       'nih_chp_05167_t1', 'a0011_t1_s03', 'nih_chp_04282_t1',
       'nih_chp_05651_t1', 'nih_chp_05192_t1', 'nih_chp_04633_t1',
       ...
       'sub-inhibition20_ses-pretraining_T1w', '1-20FORHU_t0',
       'nih_chp_02741_t1', 'sub-inhibition25_ses-pretraining_T1w',
       'a0024_t1_s03', 'a0039_t1_s03', 'nih_chp_01161_t1', 'a0022_t1_s03',
       'nih_chp_04572_t1', 'nih_chp_00901_t1'],
      dtype='object', name='Subject', length=341)

In [9]:
perc_mat_1 = get_percentile_matrix(dist_mat_1)
perc_mat_2 = get_percentile_matrix(dist_mat_2)
perc_mat_1

Subject,1-03HUBJO_t0,1-08ANDTI_t0,1-11LEBJO_t0,1-15LEHMI_t0,1-17COLMA_t0,1-18ROUCO_t0,1-20FORHU_t0,1-23CREES_t0,1-40AYMJU_t0,2-02FERMA_t0,...,sub-inhibition20_ses-pretraining_T1w,sub-inhibition21_ses-pretraining_T1w,sub-inhibition22_ses-pretraining_T1w,sub-inhibition23_ses-pretraining_T1w,sub-inhibition24_ses-pretraining_T1w,sub-inhibition25_ses-pretraining_T1w,sub-inhibition26_ses-pretraining_T1w,sub-inhibition28_ses-pretraining_T1w,sub-inhibition29_ses-pretraining_T1w,sub-inhibition31_ses-pretraining_T1w
Subject,,,,,,,,,,,,,,,,,,,,,
1-03HUBJO_t0,0.000000,93.255132,50.439883,94.134897,56.011730,36.363636,97.653959,43.401760,66.275660,86.803519,...,95.014663,68.914956,43.988270,89.736070,96.774194,99.120235,90.322581,70.674487,96.480938,68.035191
1-08ANDTI_t0,92.668622,0.000000,2.052786,19.354839,88.563050,96.774194,25.806452,60.997067,9.677419,80.351906,...,82.111437,42.228739,28.739003,83.870968,64.222874,49.560117,95.014663,91.788856,7.038123,89.149560
1-11LEBJO_t0,46.041056,3.519062,0.000000,49.560117,80.645161,83.284457,39.882698,34.604106,7.917889,64.222874,...,97.360704,49.266862,38.709677,67.155425,65.982405,87.096774,97.067449,95.307918,73.020528,72.140762
1-15LEHMI_t0,85.923754,23.753666,47.800587,0.000000,8.797654,64.222874,39.002933,31.964809,5.571848,31.671554,...,22.580645,69.208211,63.049853,46.627566,19.941349,18.768328,31.378299,64.516129,43.695015,85.337243
1-17COLMA_t0,71.260997,95.601173,90.615836,13.489736,0.000000,0.293255,74.486804,54.545455,65.395894,92.375367,...,26.686217,94.721408,95.894428,80.645161,82.697947,69.501466,20.821114,90.615836,87.683284,74.193548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sub-inhibition25_ses-pretraining_T1w,97.947214,49.853372,84.457478,18.768328,43.988270,66.862170,26.979472,22.287390,2.052786,42.815249,...,39.882698,74.780059,77.419355,82.111437,41.935484,0.000000,3.225806,85.630499,22.287390,93.255132
sub-inhibition26_ses-pretraining_T1w,93.255132,97.653959,98.826979,35.483871,10.557185,16.422287,50.733138,78.005865,72.727273,24.926686,...,43.401760,88.269795,90.322581,69.794721,77.126100,1.759531,0.000000,75.659824,68.914956,44.868035
sub-inhibition28_ses-pretraining_T1w,4.692082,12.903226,14.662757,1.759531,4.985337,4.398827,11.436950,7.624633,4.105572,6.451613,...,1.173021,4.105572,14.076246,1.173021,1.466276,7.331378,5.865103,0.000000,1.759531,3.225806


In [11]:
get_distance(perc_mat_1, perc_mat_2)

0.0

In [14]:
latent_space_distance(embeddings1, embeddings2)

0.0

## Apply the method to collected data

In [15]:
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run2/pca_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

0.12437126816237146

In [16]:
# for a good net
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/13-15-48/cingulate_ACCpatterns_embeddings/full_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/16-39-35/cingulate_ACCpatterns_embeddings/full_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

0.8780202783181796

In [17]:
# for a bad net
path1 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/11-41-24/cingulate_ACCpatterns_embeddings/full_embeddings.csv"
path2 = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/15-20-30/cingulate_ACCpatterns_embeddings/full_embeddings.csv"

embeddings1 = pd.read_csv(path1, index_col=0)
embeddings2 = pd.read_csv(path2, index_col=0)

latent_space_distance(embeddings1, embeddings2)

2.5916841435669835

In [19]:
# compute the space distances for some model types and functions

paths1 = ["/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run1/pca_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/13-15-48/cingulate_ACCpatterns_embeddings/full_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/11-41-24/cingulate_ACCpatterns_embeddings/full_embeddings.csv"]
paths2 = ["/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/pca/30/Run2/pca_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/convnet/no_foldlabel_30/16-39-35/cingulate_ACCpatterns_embeddings/full_embeddings.csv",
          "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/densenet2/no_foldlabel_4/15-20-30/cingulate_ACCpatterns_embeddings/full_embeddings.csv"]

idx_names = ['same', 'pca', 'good_net', 'bad_net', 'random']

functions = [np.sqrt, custom_ln, custom_identity]
functions_names = ['sqrt', 'custom_ln', 'custom_identity']

n_rows = len(idx_names)
n_fct = len(functions) + 1

results_df = pd.DataFrame(np.zeros((n_rows, n_fct)), columns=['original_nn']+functions_names, 
                          index=idx_names)


for j,fct in enumerate(functions):
    print(functions_names[j])
    for i in range(len(paths1)):
        embs1 = pd.read_csv(paths1[i], index_col=0)
        embs2 = pd.read_csv(paths2[i], index_col=0)
        value = latent_space_distance(embs1, embs2, fct=fct)
        print(i+1,value)
        results_df[functions_names[j]][idx_names[i+1]] = value

# random embeddings
random_embs_1 = pd.DataFrame(np.random.normal(size=(341,30)))
random_embs_2 = pd.DataFrame(np.random.normal(size=(341,30)))

for j,fct in enumerate(functions):
    print(functions_names[j])
    # random spaces
    value = latent_space_distance(random_embs_1, random_embs_2, fct=fct)
    print('random', value)
    results_df[functions_names[j]]['random'] = value
    # almost identical spaces (just a test)
    value = latent_space_distance(random_embs_1, -random_embs_1, fct=fct)
    print('identical', value)
    results_df[functions_names[j]]['same'] = value

results_df

sqrt
1 0.12437126816237146
2 0.8780202783181796
3 2.5916841435669835
custom_ln
1 0.045858203556882714
2 0.3156458029414119
3 0.9022828466349324
custom_identity
1 1.5006170838913047
2 10.79147411666163
3 32.28179608699433
sqrt
random 2.6286089697945374
identical 0.0
custom_ln
random 0.910445897844433
identical 0.0
custom_identity
random 32.928964346933775
identical 0.0


,original_nn,sqrt,custom_ln,custom_identity
same,0.0,0.000000,0.000000,0.000000
pca,0.0,0.124371,0.045858,1.500617
good_net,0.0,0.878020,0.315646,10.791474
bad_net,0.0,2.591684,0.902283,32.281796
random,0.0,2.628609,0.910446,32.928964


In [20]:
backbones = ['convnet', 'densenet2']
conds = ['foldlabel_30', 'foldlabel_4', 'no_foldlabel_30', 'no_foldlabel_4']


for backbone in backbones:
    for cond in conds:
        
        print(backbone, cond)

        main_path = f"/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/{backbone}/{cond}"
        save_path = f"/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn/{backbone}_{cond}"

        models = [model for model in os.listdir(main_path)
                if os.path.isdir(os.path.join(main_path, model))]
        models.sort()

        print(models)

        functions = [custom_identity, np.sqrt, custom_ln, custom_ln_100]
        functions_names = ['identity', 'sqrt', 'ln', 'ln_100']

        for i in range(len(models)-1):
            for j in range(i+1, len(models)):
                model_path1 = os.path.join(main_path, models[i])
                model_path2 = os.path.join(main_path, models[j])
                
                embs1 = pd.read_csv(model_path1+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)
                embs2 = pd.read_csv(model_path2+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)

                values = {}

                dist_mat1 = get_distance_matrix(embs1)
                dist_mat2 = get_distance_matrix(embs2)

                perc_mat1 = get_percentile_matrix(dist_mat1)
                perc_mat2 = get_percentile_matrix(dist_mat2)

                for k,fct in enumerate(functions):
                    values[functions_names[k]] = get_distance(perc_mat1, perc_mat2, fct=fct)
                    print(models[i], models[j], functions_names[k], values[functions_names[k]])
                
                with open(os.path.join(save_path, models[i]+'_'+models[j]+"_nne.json"), 'w') as file:
                    json.dump(values, file)

convnet foldlabel_30
['09-02-52', '11-19-12', '11-55-03', '13-51-11', '2022-07-05']
09-02-52 11-19-12 identity 15.558891986322648
09-02-52 11-19-12 sqrt 1.274889006104276
09-02-52 11-19-12 ln 0.4606264143339371
09-02-52 11-19-12 ln_100 0.1000237549820549
09-02-52 11-55-03 identity 13.79869035522984
09-02-52 11-55-03 sqrt 1.1357549005930276
09-02-52 11-55-03 ln 0.412594529148013
09-02-52 11-55-03 ln_100 0.0895937636362262
09-02-52 13-51-11 identity 14.386940766226097
09-02-52 13-51-11 sqrt 1.1834696944105043
09-02-52 13-51-11 ln 0.42941498951303947
09-02-52 13-51-11 ln_100 0.09324628019602786
09-02-52 2022-07-05 identity 14.685408773534007
09-02-52 2022-07-05 sqrt 1.203639112938913
09-02-52 2022-07-05 ln 0.4347753502133012
09-02-52 2022-07-05 ln_100 0.09441026773259524
11-19-12 11-55-03 identity 14.662822168999499
11-19-12 11-55-03 sqrt 1.2134303202140948
11-19-12 11-55-03 ln 0.4430672208610318
11-19-12 11-55-03 ln_100 0.09621082456607771
11-19-12 13-51-11 identity 15.326751323728617
11

In [24]:
# sum up the nn distances into a csv

fct = 'sqrt'

saved_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn"
models = ['densenet2_foldlabel_30', 'densenet2_no_foldlabel_30', 'densenet2_foldlabel_4', 'densenet2_no_foldlabel_4',
'convnet_foldlabel_30', 'convnet_no_foldlabel_30', 'convnet_foldlabel_4', 'convnet_no_foldlabel_4']

mod_conds = [folder for folder in os.listdir(saved_path) if os.path.isdir(os.path.join(saved_path,folder))]

nne_mean = np.zeros((len(mod_conds)//2, 2))

for folder in mod_conds:
    j = 0 if 'convnet' in folder else 1
    if "no" in folder:
        i = 2
    else:
        i = 0
    if '4' in folder:
        i += 1
    
    func = []
    for file in os.listdir(os.path.join(saved_path,folder)):
        if 'nne' in file:

            with open(os.path.join(saved_path, folder, file), 'r') as file_txt:
                file_content = json.load(file_txt)
            func.append(file_content[fct])

    nne_mean[i,j] = np.mean(func)

df = pd.DataFrame(nne_mean, columns=['convnet', 'densenet2'], index=['foldlabel_30', 'foldlabel_4', 'no_foldlabel_30', 'no_foldlabel_4'])
df.to_csv(saved_path+f'/nne_{fct}.csv')
df

,convnet,densenet2
foldlabel_30,1.194958,1.656831
foldlabel_4,2.238691,2.059200
no_foldlabel_30,0.903307,1.185040
no_foldlabel_4,2.438428,2.153048


### Pointnet nn

In [10]:
main_path = "/neurospin/dico/agaudin/Runs/04_pointnet/Output/pointnet/file_no_30_drop_rate=0.05"
save_path = main_path + "/#nn"

models = [model for model in os.listdir(main_path)
          if ((os.path.isdir(os.path.join(main_path, model)))
              & ("#" not in model))]
models.sort()

print(models)

functions = [custom_ln]
functions_names = ['ln']

for i in range(len(models)-1):
    for j in range(i+1, len(models)):
        model_path1 = os.path.join(main_path, models[i])
        model_path2 = os.path.join(main_path, models[j])
        
        embs1 = pd.read_csv(model_path1+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)
        embs2 = pd.read_csv(model_path2+"/cingulate_ACCpatterns_embeddings/full_embeddings.csv", index_col=0)

        values = {}

        dist_mat1 = get_distance_matrix(embs1)
        dist_mat2 = get_distance_matrix(embs2)

        perc_mat1 = get_percentile_matrix(dist_mat1)
        perc_mat2 = get_percentile_matrix(dist_mat2)

        for k,fct in enumerate(functions):
            values[functions_names[k]] = get_distance(perc_mat1, perc_mat2, fct=fct)
            print(models[i], models[j], functions_names[k], values[functions_names[k]])
        
        with open(os.path.join(save_path, models[i]+'_'+models[j]+"_nne.json"), 'w') as file:
            json.dump(values, file)

['10-28-55', '13-53-39', 'no_30_drop_rate=0.05']
10-28-55 13-53-39 ln 0.4858782750668692
10-28-55 no_30_drop_rate=0.05 ln 0.4884125900976653
13-53-39 no_30_drop_rate=0.05 ln 0.3625499147244931


In [11]:
main_path = "/neurospin/dico/agaudin/Runs/04_pointnet/Output/pointnet"
folders = [folder for folder in os.listdir(main_path)
           if 'file' in folder]

for folder in folders:
    path = os.path.join(main_path, folder) + '/#nn'
    mean_nn = []
    for filename in os.listdir(path):
        if 'nne' in filename:
            with open(path+'/'+filename, 'r') as file:
                nne = json.load(file)
                mean_nn.append(nne['ln'])
    mean_nn = np.mean(mean_nn)
    with open(path+'/mean_nn.txt', 'w') as save_file:
        save_file.write(str(mean_nn))

## Compare the results 

In [25]:
saved_path = "/neurospin/dico/agaudin/Runs/03_monkeys/Output/analysis_folders/#performance_comparisons/nn"

id_df = pd.read_csv(saved_path+'/nne_identity.csv', index_col=0)
sqrt_df = pd.read_csv(saved_path+'/nne_sqrt.csv', index_col=0)
ln_df = pd.read_csv(saved_path+'/nne_ln.csv', index_col=0)
ln_100_df = pd.read_csv(saved_path+'/nne_ln_100.csv', index_col=0)


nn_df = pd.read_csv(saved_path+'/mean_median_nn.csv', index_col=0)

print("id")
print(id_df)
print()

print("sqrt")
print(sqrt_df)
print()

print("ln")
print(ln_df)
print()

print("ln_100")
print(ln_100_df)
print()

print("nn")
print(nn_df)

id
                   convnet  densenet2
foldlabel_30     14.522920  20.158161
foldlabel_4      27.855698  25.365510
no_foldlabel_30  11.108282  14.629181
no_foldlabel_4   30.402163  26.989333

sqrt
                  convnet  densenet2
foldlabel_30     1.194958   1.656831
foldlabel_4      2.238691   2.059200
no_foldlabel_30  0.903307   1.185040
no_foldlabel_4   2.438428   2.153048

ln
                  convnet  densenet2
foldlabel_30     0.433729   0.596620
foldlabel_4      0.783451   0.731105
no_foldlabel_30  0.324525   0.422612
no_foldlabel_4   0.849684   0.750240

ln_100
                  convnet  densenet2
foldlabel_30     0.094183   0.129554
foldlabel_4      0.170124   0.158758
no_foldlabel_30  0.070470   0.091769
no_foldlabel_4   0.184507   0.162913

nn
                   convnet  densenet2
foldlabel_30      3.189150  10.630499
foldlabel_4      41.788856  26.906158
no_foldlabel_30   1.282991   2.492669
no_foldlabel_4   39.809384  26.722874


In [30]:
dfs = [id_df, sqrt_df, ln_df, ln_100_df, nn_df]

cond = 'no_foldlabel_4'

for df in dfs:
    ratio = df['convnet'][cond]/df['densenet2'][cond]
    print(ratio)

1.1264510312765748
1.132547119516568
1.1325498477557974
1.1325498477557976
1.4897119341563791


In [31]:
dfs = [id_df, sqrt_df, ln_df, ln_100_df, nn_df]

model = 'convnet'
cond1 = 'no_foldlabel_30'
cond2 = 'foldlabel_30'

for df in dfs:
    ratio = df[model][cond1]/df[model][cond2]
    print(ratio)

0.764879374203713
0.7559320789320302
0.7482196376096077
0.7482196376096076
0.4022988505747127


fct has almost no impact on the predictions: the absolute values might be different, but the ratio remains the same.